# 에브리타임 강의평가 크롤러
subject_list.xlsx 를 열어서 형식 확인
'과목명' '과목코드'가 있는데,
이 중에서 '과목명'만 check하면 됨.


In [2]:
# 패키지 설치
!pip install selenium
!pip install pandas openpyxl
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import csv
import random
from bs4 import BeautifulSoup



Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
# 엑셀 파일 경로 (생략 가능)
file_path = './subject_list.xlsx'

# 엑셀 파일을 읽어와 데이터프레임으로 변환 (생략 가능)
df = pd.read_excel(file_path, engine='openpyxl')

# 데이터프레임 출력 (생략 가능)
print(df)


                          과목명          과목코드
0                      기계공학개론       400.013
1                      산업공학개론       400.015
2                   전기.정보공학개론      400.019A
3                      재료공학개론       400.020
4                    건설환경공학개론       400.022
5                    화학생물공학개론       400.023
6                   에너지자원공학개론       400.024
7                     인공지능 개론  M2177.008600
8             정보 보호 및 블록체인 개론  M2177.008700
9              VR/AR의 개론 및 실습  M2177.007800
10                   공학 전공 탐색  M2177.008100
11                     기술과 창업       400.212
12                    공학기술과사회       400.310
13                       공학경제       400.316
14                     공학수학 3       400.003
15                    딥러닝의 기초  M2177.004300
16  머신러닝을 위한 기초 수학 및 프로그래밍 실습  M2177.005800
17                     창의공학설계       400.018
18               다학제 창의적 제품개발  M2177.002300
19                  로봇인공지능만들기  M2177.002600
20               블록체인의 실무응용 1  M2177.006600
21                     공학수학 1   

In [17]:
# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument('--disable-blink-features=AutomationControlled') # 봇 방지 우회 옵션

# 크롬 드라이버 생성
driver = webdriver.Chrome(options=chrome_options)

# 페이지 로딩이 완료될 때 까지 기다리는 코드
driver.implicitly_wait(3)

time.sleep(10)

# 사이트 접속하기
driver.get(url='https://everytime.kr/login')




chrome webdriver 새 창에 에브리타임 로그인 창이 나오게 될 것입니다. 로그인하세요!
새로 열린 chrome 창을 닫지 말고 아래 코드들을 작동해 주십시오.

##################################################################

또한 이 부분에서 keyword = '(찾고자 하는 과목명 입력)' 을 설정하면
 몇 분 후에 csv 파일로 과목명으로 자동 정리됩니다.

##################################################################

그리고 반복적으로 짧은 시간 내에 지속적으로 크롤링을 할 경우 에브리타임 측에서 이용한 계정을 영구적으로 차단할 수 있습니다.
따라서 긴 term을 두고 crawling을 하길 바랍니다.

원리는 다음과 같습니다.
1. 에브리타임은 검색어를 입력하면 그 과목만 출력되지 않고 검색어의 단어를 포함하는 과목을 모두 출력합니다.
2. 에브리타임은 강의별로 강의평가가 있으면 별점이 0점이 아닙니다. 

두 가지 조건에 부합하는 주소를 찾아서 교수 / OO년 O학기 수강자 / 평점 / text 를 parsed_articles의 형태로 추출합니다.

In [34]:
# 새로운 탭 열기
driver.execute_script("window.open('');")
time.sleep(2)  # 탭이 열릴 시간을 주기 위해 잠시 대기

# 새로운 탭으로 전환
driver.switch_to.window(driver.window_handles[1])

random_time = random.uniform(6, 10)
time.sleep(random_time)


##################### 검색할 키워드 ################################ 
keyword = '통계학실험'
#keyword = df['과목명'][1]
####################################################################



# 새로운 탭에서 특정 URL로 이동
driver.get(f'https://everytime.kr/lecture/search?keyword={keyword}&condition=name')

# 페이지 로딩 대기
time.sleep(10)

# 자바스크립트를 사용하여 페이지 끝까지 스크롤
last_height = driver.execute_script("return document.body.scrollHeight")

while True:

    random_time = random.uniform(2, 4)

    # 페이지 끝까지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 스크롤 후 로딩 대기
    time.sleep(random_time)
    
    # 새로운 높이 계산
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # 더 이상 스크롤할 수 없으면 종료
    if new_height == last_height:
        
        break
    
    last_height = new_height

# 스크롤 완료 후 대기
time.sleep(3)
print("scroll completed")

# /html/body/div/div/div[2] 요소 찾기
element = driver.find_element(By.XPATH, '/html/body/div/div/div[2]')

# 요소의 HTML 소스 가져오기
element_html = element.get_attribute('outerHTML')

scroll completed


In [35]:
# BeautifulSoup을 사용하여 HTML 소스를 파싱
soup = BeautifulSoup(element_html, 'html.parser')

# 파싱된 HTML 출력 (보기 쉽게 포맷팅)
formatted_html = soup.prettify()

# 결과 저장 리스트
results = []
# 모든 <a> 태그 순회
for lecture in soup.find_all('a', class_='lecture'):
    # 조건 1: highlight 텍스트와 name 텍스트가 동일해야 함
    name_div = lecture.find('div', class_='name')
    if name_div:
        highlight_span = name_div.find('span', class_='highlight')
        if highlight_span:
            highlight_text = highlight_span.get_text()  # highlight 텍스트
            name_text = name_div.get_text(strip=True)  # name 전체 텍스트
            if highlight_text == name_text:  # name 텍스트와 highlight 텍스트가 동일
                # 조건 2: style="width: X%;" 확인
                rate_span = lecture.find('span', class_='on')
                if rate_span and 'style' in rate_span.attrs:
                    width_value = rate_span['style']
                    if 'width:' in width_value and not width_value.strip().startswith('width: 0%'):
                        # 조건 만족 시 href와 professor 추출
                        href = lecture.get('href')
                        professor_div = lecture.find('div', class_='professor')
                        professor_name = professor_div.get_text(strip=True) if professor_div else "Unknown"
                        if href:
                            results.append({'url': href, 'professor': professor_name})

# 결과 출력
print("Filtered Results:")
for result in results:
    print(f"URL: {result['url']}, Professor: {result['professor']}")

Filtered Results:
URL: /lecture/view/2766845, Professor: 김지수
URL: /lecture/view/2385929, Professor: 이권상
URL: /lecture/view/2385928, Professor: PARK JUN YONG
URL: /lecture/view/2385927, Professor: 박건웅
URL: /lecture/view/2020033, Professor: 김미정
URL: /lecture/view/2020032, Professor: 한상미
URL: /lecture/view/1947732, Professor: 정성규
URL: /lecture/view/1624845, Professor: 이상열
URL: /lecture/view/847314, Professor: 이영조
URL: /lecture/view/847299, Professor: 임채영
URL: /lecture/view/847296, Professor: 박병욱
URL: /lecture/view/615329, Professor: 임요한
URL: /lecture/view/609040, Professor: 김용대
URL: /lecture/view/609028, Professor: 박지영
URL: /lecture/view/609027, Professor: 장원철
URL: /lecture/view/609024, Professor: 원중호
URL: /lecture/view/602715, Professor: 정상아
URL: /lecture/view/602714, Professor: 이선순
URL: /lecture/view/602713, Professor: 이재용


In [36]:
# 결과 HTML 저장 리스트
collected_html = []
# 새로운 탭에서 작업하기
for result in results:
    # 새로운 탭 열기
    driver.execute_script("window.open('');")
    time.sleep(2)  # 탭이 열릴 시간을 주기 위해 잠시 대기

    # 새로운 탭으로 전환
    driver.switch_to.window(driver.window_handles[1])

    random_time = random.uniform(5, 10)
    time.sleep(random_time)

    # URL 생성 및 이동
    url_to_visit = f"https://everytime.kr{result['url']}?tab=article"
    driver.get(url_to_visit)

    random_time = random.uniform(5, 10)
    time.sleep(random_time)  # 페이지 로딩 대기
    # 자바스크립트를 사용하여 특정 요소 끝까지 스크롤
    element_selector = "/html/body/div/div/div[2]/div/div[2]"  # 대상 요소
    try:
        element = driver.find_element(By.XPATH, element_selector)
        last_height = driver.execute_script("return arguments[0].scrollHeight;", element)

        while True:
            random_time = random.uniform(2, 5)  # 랜덤 대기시간
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", element)  # 스크롤 실행
            time.sleep(random_time)

            # 새로운 높이 계산
            new_height = driver.execute_script("return arguments[0].scrollHeight;", element)
            if new_height == last_height:
                break  # 더 이상 스크롤할 수 없으면 종료
            last_height = new_height

        print(f"Scrolling completed for {url_to_visit}")

        # 요소의 HTML 소스 가져오기
        element_html = element.get_attribute('outerHTML')
        collected_html.append({
            'url': url_to_visit,
            'professor': result['professor'],
            'html': element_html
        })

    except Exception as e:
        print(f"Error while processing {url_to_visit}: {e}")

    # 현재 탭 닫기
    driver.close()
    time.sleep(2)

    # 원래 창으로 복귀
    driver.switch_to.window(driver.window_handles[0])

# 결과 확인
for item in collected_html:
    print(f"Collected HTML for {item['url']} - Professor: {item['professor']}")

Scrolling completed for https://everytime.kr/lecture/view/2766845?tab=article
Scrolling completed for https://everytime.kr/lecture/view/2385929?tab=article
Scrolling completed for https://everytime.kr/lecture/view/2385928?tab=article
Scrolling completed for https://everytime.kr/lecture/view/2385927?tab=article
Scrolling completed for https://everytime.kr/lecture/view/2020033?tab=article
Scrolling completed for https://everytime.kr/lecture/view/2020032?tab=article
Scrolling completed for https://everytime.kr/lecture/view/1947732?tab=article
Scrolling completed for https://everytime.kr/lecture/view/1624845?tab=article
Scrolling completed for https://everytime.kr/lecture/view/847314?tab=article
Scrolling completed for https://everytime.kr/lecture/view/847299?tab=article
Scrolling completed for https://everytime.kr/lecture/view/847296?tab=article
Scrolling completed for https://everytime.kr/lecture/view/615329?tab=article
Scrolling completed for https://everytime.kr/lecture/view/609040?tab

In [37]:
# CSV 파일 저장 경로
output_csv = f"{keyword}_parsed_articles.csv"

# CSV 저장
with open(output_csv, mode="w", encoding="utf-8-sig", newline="") as file:  # 인코딩을 utf-8-sig로 설정
    writer = csv.writer(file)
    
    # CSV 헤더 작성
    writer.writerow(["Professor", "Semester", "Rating", "Text"])
    
    # 각 collected_html 항목 처리
    for item in collected_html:
        professor_name = item['professor']
        html_content = item['html']
        
        # BeautifulSoup로 HTML 파싱
        soup = BeautifulSoup(html_content, "html.parser")
        
        # <article> 태그 리스트 추출
        articles = soup.find_all("div", class_="article")
        
        for article in articles:
            # Semester 정보 추출
            semester = article.find("span", class_="semester")
            semester_text = semester.get_text(strip=True) if semester else "Unknown"
            
            # Rating 정보 추출
            rating = article.find("span", class_="on")
            rating_style = rating.get("style") if rating else "width: 0%;"
            rating_value = rating_style.split(":")[1].strip().rstrip(";") if "width:" in rating_style else "0%"
            
            # Text 정보 추출
            text_div = article.find("div", class_="text")
            text_content = text_div.get_text(strip=True) if text_div else "No text"
            
            # CSV에 작성
            writer.writerow([professor_name, semester_text, rating_value, text_content])

print(f"CSV saved to {output_csv}")


CSV saved to 통계학실험_parsed_articles.csv
